# MapReduce

In [3]:
import os
from nose.tools import assert_equal, assert_true

In [4]:
os.environ['PYTHONIOENCODING'] = 'latin-1'

In [5]:
python_io_encoding = ! echo $PYTHONIOENCODING
assert_equal(python_io_encoding.s, 'latin-1')

## Mapper

In [27]:
%%writefile mapper.py
#!/usr/bin/env python3

import sys

for line in sys.stdin:
    if not(line.startswith('Year')):
        aline = line.split(",")
        print(aline[16]+'\t'+aline[13])

Overwriting mapper.py


In [28]:
! chmod u+x mapper.py

In [29]:
! head -n 50 2001.csv > 2001.csv.head
map_out_head = ! ./mapper.py < 2001.csv.head
print('\n'.join(map_out_head))

BWI	60
BWI	64
BWI	80
BWI	66
BWI	62
BWI	61
BWI	61
BWI	60
BWI	52
BWI	62
BWI	62
BWI	55
BWI	60
BWI	61
BWI	63
PHL	53
PHL	54
PHL	55
PHL	53
PHL	50
PHL	NA
PHL	57
PHL	48
PHL	56
PHL	55
PHL	55
PHL	55
PHL	55
PHL	49
PHL	75
PHL	49
PHL	50
PHL	49
PHL	NA
PHL	46
PHL	NA
PHL	51
PHL	53
PHL	52
PHL	52
PHL	54
PHL	56
PHL	55
PHL	51
PHL	49
PHL	49
CLT	82
CLT	82
CLT	78


In [30]:
assert_equal(
    map_out_head,
    ['BWI\t60','BWI\t64','BWI\t80','BWI\t66','BWI\t62','BWI\t61',
     'BWI\t61','BWI\t60','BWI\t52','BWI\t62','BWI\t62','BWI\t55',
     'BWI\t60','BWI\t61','BWI\t63','PHL\t53','PHL\t54','PHL\t55',
     'PHL\t53','PHL\t50','PHL\tNA','PHL\t57','PHL\t48','PHL\t56',
     'PHL\t55','PHL\t55','PHL\t55','PHL\t55','PHL\t49','PHL\t75',
     'PHL\t49','PHL\t50','PHL\t49','PHL\tNA','PHL\t46','PHL\tNA',
     'PHL\t51','PHL\t53','PHL\t52','PHL\t52','PHL\t54','PHL\t56',
     'PHL\t55','PHL\t51','PHL\t49','PHL\t49','CLT\t82','CLT\t82',
     'CLT\t78']
    )

## Reducer

In [40]:
%%writefile reducer.py
#!/usr/bin/env python3

import sys

sep = '\t'

minmax = dict()

for line in sys.stdin:
    line = line.strip()
    airport, airtime = line.split('\t', 1)
    
    if airtime!='NA':
        at = int(airtime)
   
    if airport not in minmax.keys():
        minmax[airport] = [float("inf"),-float("inf")]
    
    if at>minmax[airport][1]:
        minmax[airport][1]=at
                
    if at<minmax[airport][0]:
        minmax[airport][0]=at

for i in sorted(minmax):
    print(i+'\t'+str(minmax[i][0])+'\t'+str(minmax[i][1]))

Overwriting reducer.py


In [41]:
! chmod u+x reducer.py

In [42]:
red_head_out = ! ./mapper.py < 2001.csv.head | sort -n -k 1 | ./reducer.py
print('\n'.join(red_head_out))

BWI	52	80
CLT	78	82
PHL	46	75


In [45]:
mapred_out = ! ./mapper.py < 2001.csv | sort -n -k 1 | ./reducer.py
print('\n'.join(mapred_out[:10]))

ABE	16	180
ABI	28	85
ABQ	15	264
ACT	19	81
ACY	33	33
ADQ	32	67
AKN	12	54
ALB	23	360
AMA	30	130
ANC	23	428


In [46]:
assert_equal(len(mapred_out), 231)
assert_equal(mapred_out[:5], ['ABE\t16\t180', 'ABI\t28\t85', 'ABQ\t15\t264', 'ACT\t19\t81', 'ACY\t33\t33'])
assert_equal(mapred_out[-5:], ['TYS\t11\t177', 'VPS\t28\t123', 'WRG\t5\t38', 'XNA\t33\t195', 'YAK\t28\t72'])

## HDFS: Reset

In [47]:
! $HADOOP_HOME/sbin/stop-dfs.sh
! $HADOOP_HOME/sbin/stop-yarn.sh

17/08/18 22:28:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: no namenode to stop
localhost: no datanode to stop
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: no secondarynamenode to stop
17/08/18 22:28:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
stopping yarn daemons
no resourcemanager to stop
localhost: no nodemanager to stop
no proxyserver to stop


In [48]:
! rm -rf /tmp/*

In [49]:
! echo "Y" | $HADOOP_HOME/bin/hdfs namenode -format 2> /dev/null

Formatting using clusterid: CID-cdfdc43e-b89a-420f-9f8d-3c75fb28f2b1


In [50]:
!$HADOOP_HOME/etc/hadoop-env.sh
!$HADOOP_HOME/sbin/start-dfs.sh
!$HADOOP_HOME/sbin/start-yarn.sh

/bin/sh: /usr/local/Cellar/hadoop/2.8.0/etc/hadoop/hadoop-env.sh: No such file or directory
17/08/18 22:29:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.8.0/libexec/logs/hadoop-Constance-namenode-host-173-230-8-252.ilcuarb.clients.pavlovmedia.com.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.8.0/libexec/logs/hadoop-Constance-datanode-host-173-230-8-252.ilcuarb.clients.pavlovmedia.com.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.8.0/libexec/logs/hadoop-Constance-secondarynamenode-host-173-230-8-252.ilcuarb.clients.pavlovmedia.com.out
17/08/18 22:29:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
starting yarn daemons
sta

In [52]:
! $HADOOP_HOME/bin/hdfs dfsadmin -safemode leave

17/08/18 22:30:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Safe mode is OFF


## HDFS: Create directory

In [ ]:
!$HADOOP_HOME/bin/hdfs dfs -rm -r -f 

In [53]:
!$HADOOP_HOME/bin/hdfs dfs -mkdir /user
!$HADOOP_HOME/bin/hdfs dfs -mkdir /user/data_scientist

17/08/18 22:46:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
17/08/18 22:46:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [54]:
ls_user = ! $HADOOP_HOME/bin/hdfs dfs -ls /user/
print('\n'.join(ls_user))

17/08/18 22:46:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
drwxr-xr-x   - Constance supergroup          0 2017-08-18 22:46 /user/data_scientist


In [55]:
assert_true('/user/data_scientist' in ls_user.s)

In [73]:
!$HADOOP_HOME/bin/hdfs dfs -mkdir /user/data_scientist/wc
!$HADOOP_HOME/bin/hdfs dfs -mkdir /user/data_scientist/wc/in

17/08/18 23:14:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
mkdir: `/user/data_scientist/wc': File exists
17/08/18 23:15:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
mkdir: `/user/data_scientist/wc/in': File exists


In [77]:
ls_wc = !$HADOOP_HOME/bin/hdfs dfs -ls /user/data_scientist/wc
print('\n'.join(ls_wc))

17/08/18 23:21:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
drwxr-xr-x   - Constance supergroup          0 2017-08-18 22:48 /user/data_scientist/wc/in


## HDFS: Copy

In [78]:
!$HADOOP_HOME/bin/hdfs dfs -put 2001.csv /user/data_scientist/wc/in

17/08/18 23:21:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [83]:
!$HADOOP_HOME/bin/hdfs dfs -ls /user/data_scientist/wc/in

17/08/18 23:25:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
-rw-r--r--   1 Constance supergroup  600411462 2017-08-18 23:21 /user/data_scientist/wc/in/2001.csv


## Python Hadoop Streaming

In [87]:
%%bash
# Delete output directory (if it exists)
$HADOOP_HOME/bin/hdfs dfs -rm -r -f wc/out

# Grab current streaming lib jar filename
streaming_file=$(ls $HADOOP_HOME/libexec/share/hadoop/tools/lib/hadoop-streaming*)

# Run the Map Reduce task within Hadoop
$HADOOP_HOME/bin/hadoop jar $streaming_file \
    -files mapper.py,reducer.py -input /user/data_scientist/wc/in \
    -output wc/out -mapper "python mapper.py" -reducer "python reducer.py"\
    -cmdenv PYTHONIOENCODING='latin-1'

Deleted wc/out
packageJobJar: [/var/folders/kg/sql8zr092nz03ztkw2prx_z80000gn/T/hadoop-unjar7640413220499755904/] [] /var/folders/kg/sql8zr092nz03ztkw2prx_z80000gn/T/streamjob2576565045769441957.jar tmpDir=null


17/08/18 23:36:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
17/08/18 23:36:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
17/08/18 23:36:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/08/18 23:36:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/08/18 23:36:24 INFO mapred.FileInputFormat: Total input files to process : 1
17/08/18 23:36:25 INFO mapreduce.JobSubmitter: number of splits:5
17/08/18 23:36:25 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1503113388432_0002
17/08/18 23:36:26 INFO impl.YarnClientImpl: Submitted application application_1503113388432_0002
17/08/18 23:36:26 INFO mapreduce.Job: The url to track the job: http://host-173-230-8-252.ilcuarb.clients.pavlovmedia.com:8088/proxy/application_1503113388432_0002/
17/08/18 23:36:26 INFO mapr

In [94]:
ls_wc_out = !$HADOOP_HOME/bin/hdfs dfs -ls wc/out
print('\n'.join(ls_wc_out))

17/08/18 23:55:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   1 Constance supergroup          0 2017-08-18 23:38 wc/out/_SUCCESS
-rw-r--r--   1 Constance supergroup       2464 2017-08-18 23:38 wc/out/part-00000


In [95]:
assert_true('wc/out/_SUCCESS' in ls_wc_out.s)
assert_true('wc/out/part-00000' in ls_wc_out.s)

In [89]:
stream_out = ! $HADOOP_HOME/bin/hdfs dfs -cat wc/out/part-00000
print('\n'.join(stream_out[:10]))

17/08/18 23:39:43 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ABE	16	180
ABI	28	85
ABQ	15	264
ACT	19	81
ACY	33	33
ADQ	32	67
AKN	12	54
ALB	23	360
AMA	30	130


## Cleanup

In [101]:
!$HADOOP_HOME/bin/hdfs dfs -rm -r -f -skipTrash wc/out

17/08/19 00:00:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted wc/out
